## **Instituto de Informática - UFRGS**
## Disciplina INF01017 - Aprendizado de Máquina
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
### **Trabalho 2 - Análise de agrupamentos com K-means**
<br> 

**Grupo**:
Giovani da Silva - 00305086

*** Explicações estão contidas no relatório em anexo ***


---
***Observação:*** *Este notebook é disponibilizado aos alunos como ponto de partida para o desenvolvimento do trabalho prático 2 (T2) da disciplina INF01017. Os alunos podem optar por fazer o download dos dados e realizar a análise em outro notebook (fora do Google Colab) ou outro software. Entretanto, o grupo deve se atentar em discutir os aspectos solicitados a respeito da análise de agrupamentos e interpretação dos resultados.*


---

***ENTREGA:*** A entrega deste trabalho deve ser feita enviando o **link do Google Colab** com a solução do grupo, com a opção de deixar **as saídas do notebook salvas**. Alternativamente, os grupos podem enviar o Google Colab (ou script Python) e o relatório em PDF com os resultados das análises e a interpretação dos resultados.



# Segmentação de Clientes com Algoritmo K-Means

O conjunto de dados a ser utilizado nesse trabalho foi adaptado de um problema de classificação de risco de crédito para clientes bancários. Os dados a serem analisados não possuem rótulos (classes) e possuem apenas um subconjunto previamente selecionado dos atributos utilizados para descrever as instâncias analisadas. O objetivo é realizar o processo de segmentação de clientes que solicitaram crédito bancário, que consiste em separar os clientes em grupos menores com base em características comuns entre eles. A partir destes grupos gerados (aqui denominados clusters), a empresa pode oferecer uma comunicação mais assertiva e personalizada aos seus clientes, e melhor compreender sobre os interesses e necessidades dos seus clientes ao traçar perfis de clientes (também denominados *personas*).


Os atributos disponíveis estão descritos abaixo:

*   **Age.** Idade (numérico)
*   **Credit amount.** Valor do crédito (numérico, em DM - Deutsch Mark)
*   **Duration.** Duração (numérico, em mês)
*   **Sex.** Sexo (categórico: masculino, feminino)
*   **Job.** Emprego (categórico: 0 - não qualificado e não residente, 1 - não qualificado e residente, 2 - qualificado, 3 - altamente qualificado)
*   **Housing.** Imóvel (categórico: próprio, alugado ou gratuito)
*   **Saving Account.** Poupança (categórico: pequena, moderada, bastante rico, rico)
*   **Checking Account.** Conta corrente (categórico: pequena, moderada, bastante rico, rico)
*   **Purpose.** Finalidade (categórico: carro, móveis/equipamentos, rádio/TV, eletrodomésticos, reparos, educação, negócios, férias/outros)





In [ ]:
##importando bilbiotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

Lendo os dados e visualizando a estrutura para as primeiras instâncias

In [ ]:
data = pd.read_csv("https://drive.google.com/uc?export=view&id=1Jp-y1djRI3sCT6_JTBkfLn1FOL4qMHsv",  )

In [ ]:
data.head()

A primeira coluna pode ser removida, pois é apenas um identificador (da instância ou da linha).

In [ ]:
data.drop(data.columns[0], inplace=True, axis=1)

Inspecionando o tamanho do base de dados, os tipos dos atributos e a ocorrência de valores faltantes.

In [ ]:
print("O conjunto de dados possui {} instâncias (clientes) e {} colunas (atributos).".format(data.shape[0],data.shape[1]))

In [ ]:
print("Valores faltantes (%) por atributo:\n{}".format((data.isnull().sum()/data.shape[0])*100))

Os valores faltantes ocorrem nos atributos Saving accounts e Checking account. Provavelmente são clientes que não possuem algum destes tipos de conta.

In [ ]:
print("Tipo de dado por atributo:\n{}".format(data.dtypes))

Embora o atributo Job esteja codificado como inteiro, ele possui uma interpretação **categórica**. Vamos fazer a conversão de tipo e separar os atributos em vetores de categóricos e numéricos para facilitar a análise exploratória dos dados.

In [ ]:
## Converte o atributo job para object.
data_types_dict = {'Job': object}

data = data.astype(data_types_dict)

## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(data.select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(data.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

## Separa os dados em dois dataframes, de atributos numéricos e categóricos
data_num = data[num_columns]
data_cat = data[cat_columns]

In [ ]:
data_num.describe()

A análise de agrupamento para identificar perfis de clientes será feita a partir dos dados numéricos: **Age, Credit Amount, Duration**.


Uma vez definidos os clusters, os demais atributos serão empregados para uma interpretação mais aprofundada destes clusters e dos respectivos perfis de clientes que representam.




### **Análise Exploratória dos Dados**

Nas células abaixo, vamos realizar uma análise exploratória dos dados. 

Primeiramente, vamos observar a relação entre os três atributos numéricos através de um gráfico 3D e de gráficos que traçam a relação par a par entre eles.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data["Credit amount"], data["Duration"], data["Age"])
ax.set_xlabel("Credit amount")
ax.set_ylabel("Duration")
ax.set_zlabel("Age")

In [ ]:
def scatters(data, h=None, pal=None):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8))
    sns.scatterplot(x="Credit amount",y="Duration", hue=h, palette=pal, data=data, ax=ax1)
    sns.scatterplot(x="Age",y="Credit amount", hue=h, palette=pal, data=data, ax=ax2)
    sns.scatterplot(x="Age",y="Duration", hue=h, palette=pal, data=data, ax=ax3)
    plt.tight_layout()

In [ ]:
scatters(data)

Estas análises sugerem a existência de uma correlação positiva entre Credit amount e Duration. Esta correlação é pertinente, visto que valores maiores de créditos tendem a ser pagos em um prazo maior.

Podemos observar, também, a distribuição de cada atributo numérico. 

In [ ]:
def distributions(df):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8))
    sns.distplot(df["Age"], ax=ax1)
    sns.distplot(df["Credit amount"], ax=ax2)
    sns.distplot(df["Duration"], ax=ax3)
    plt.tight_layout()

In [ ]:
distributions(data_num)

Com a análise das distribuições, percebemos de forma ainda mais clara que os atributos variam em escalas diferentes. Em algoritmos que lidam com medidas de proximidade, como é o caso do k-means, é importante normalizar os dados para que os valores dos diferentes atributos estejam em ordens de grandeza similares, e assim exerçam o mesmo impacto no aprendizado. Iremos utilizar o método StandardScaler (também chamado por padronização).

In [ ]:
scaler = StandardScaler()
data_num_std = scaler.fit_transform(data_num)
data_num_std = pd.DataFrame(data_num_std, columns=num_columns)

In [ ]:
## Visualizando novamente os dados, após padronização 
## (Apenas para entender, visualmente, que o padrão na relação dos dados não muda)
scatters(data_num_std)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_num_std["Credit amount"], data_num_std["Duration"], data_num_std["Age"])
ax.set_xlabel("Credit amount")
ax.set_ylabel("Duration")
ax.set_zlabel("Age")

### **Aplicação do K-means**

Nesta seção, o grupo deve realizar a aplicação do algoritmo [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) no dataframe `data_num_std`, seguindo os passos listados abaixo:


1.   Faça a análise do k-means para diferentes números de clusters. Sugere-se testar de 1 a 20. Crie um vetor para armazenar a soma do quadrado das distâncias das instâncias até o centróide mais próximo durante o loop (também chamado de inércia, e disponível no campo *inertia_* do objeto retornado pelo método KMeans, por exemplo, kmeans.inertia_)  
2.   Faça um gráfico da inércia (eixo y) para os diferentes valores de k (eixo x), a fim de determinar o melhor valor de k pelo método do cotovelo (Elbow method).
3.   Escolha o melhor valor de k para os dados, repetindo a execução do k-means com o k escolhido e gerando a configuração final de clusters.



In [ ]:
##1
max_clusters = 20 #@param {type:"integer"}
inertias=[]
for ii in range(1,max_clusters):
    kmeans = KMeans(n_clusters=ii, random_state=42)
    kmeans.fit(data_num_std)
    inertias.append(kmeans.inertia_)
print(inertias)

In [ ]:
##2
plt.figure()
plt.plot(range(1,max_clusters),inertias, marker='o')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
silhouette_scores = []
X = data_num_std
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(X, labels)
    silhouette_scores.append(silhouette_avg)
print(silhouette_scores)

# Plot the silhouette scores
plt.plot(range(2, 10), silhouette_scores)
plt.title('Silhouette Analysis')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
# source code: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X = data_num_std
range_n_clusters = [2,3,4,5,6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("n_clusters = %d"
        % n_clusters)
    

    ax1.set_xlabel("Valores de coeficiente da silhueta")
    ax1.set_ylabel("Cluster")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

 

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

In [ ]:
#não esta muito claro onde é o ponto de cutuvelo, parece ser entre 3 e 4

num_clusters = 3 #@param {type:"integer"}

kmeans_final = KMeans(num_clusters )
kmeans_final.fit(data_num_std)
labels=kmeans_final.labels_

A célula abaixo gera uma versão do conjunto de dados com uma coluna 'cluster' adicionada aos dados, indicando o índice do cluster ao qual foi designada cada instância. Esta versão será útil para interpretação dos resultados, buscando analisar padrões e tendências por cluster.

In [ ]:
clusters_config = pd.concat([data_num_std, data_cat, pd.DataFrame({'cluster':labels})], axis=1)
clusters_config.head()

### **Visualização do resultado do agrupamento**

Para visualizar o resultado do agrupamento em um gráfico de 2D, vamos usar a estratégia de *Principal Component Analysis* (PCA), que faz uma projeção dos dados a partir da combinação linear dos atributos (dimensões originais).

Esta transformação será realizada tanto nos dados usados no algoritmo k-means, como nos centróides dos clusters gerados pelo algoritmo.

Cada ponto será representado pelas coordenadas {PC1, PC2} (onde PC = *Principal Component*) e a cor do ponto no gráfico corresponde ao seu respectivo cluster. 

In [ ]:
from sklearn.decomposition import PCA
pca_2 = PCA(2)
pca_2_result = pca_2.fit_transform(data_num_std)

## obtém os centrois do k-means e aplica a transformação por PCA
centroids = kmeans_final.cluster_centers_
centroids_pca = pca_2.transform(centroids)

## plota a figura, colorindo os pontos de acordo com o respectivo cluster.
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=labels, cmap='viridis')
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.show()

### **Interpretação dos resultados**

A análise de agrupamentos tem como resultado a geração de **clusters** que são definidos com base na **alta similaridades entre as características** (ou atributos) **das instâncias**. Assim, usualmente o resultado do agrupamento precisa ser explorado visualmente, através de gráficos, tabelas ou estatística descritiva, e interpretado.

Nesta seção do documento, os grupos deverão implementar suas estratégias para **analisar e comparar** a distribuição dos atributos entre os clusters encontrados. **O objetivo é traçar um perfil dos clientes que se encaixa em cada cluster obtido, com base nos atributos disponíveis** (numéricos e categóricos, incluindo aqueles que não foram usados para realizar o agrupamento). Ao final da análise, os grupos devem ser capazes de realizar uma descrição de cada cluster em termos do perfil de cliente que ele melhor representa.

Por exemplo... Algum cluster está associado com pessoas mais jovens? Como está distribuído o valor de crédito em cada cluster? Existe uma proporção maior de mulheres em algum cluster? Existe alguma relação entre os clusters e o tipo de emprego, de moradia, ou de contas (poupança e correnta)? etc. Os grupos podem (e devem) usar a criatividade para realizar a análise dos dados e a interpretação dos dados. 

Sugere-se uso de recursos como média e desvio padrão, gráfico de barras, graficos de boxplot, histogramas, ou outros apropriados para análise de distribuição (de acordo com o tipo de atributo, numérico ou categórico).
 
Os resultados devem ser apresentados e discutidos no próprio notebook do Google Colab ou, alternativamente, em relatório em PDF a ser entregue junto com o notebook.




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = clusters_config
# Seleciona apenas as colunas relevantes
num_cols = ['Credit amount', 'Age', 'Duration','cluster']
df_num = df[num_cols]



# Agrupa os dados pelos clusters
grouped = df_num.groupby('cluster')

# Calcula as estatísticas descritivas
stats = grouped.describe()

# Imprime as estatísticas descritivas
print(stats)


In [ ]:
# create a dictionary where the keys are the cluster labels and the values are lists of tuples representing the feature weights of each data point in that cluster
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append([(j, w) for j, w in enumerate(kmeans.cluster_centers_[label])])

# calculate the mean weight for each feature in each cluster
cluster_feature_weights = {}
for label, tuples in clusters.items():
    cluster_feature_weights[label] = {}
    for i, attr in enumerate(['age', 'credit amount', 'duration']):
        feature_sum = 0
        for tpl in tuples:
            feature_sum += tpl[i][1]
        cluster_feature_weights[label][attr] = feature_sum / len(tuples)

In [ ]:
n = 3  # top n features
top_features = {}
for label, feature_weights in cluster_feature_weights.items():
    sorted_features = sorted(feature_weights.items(), key=lambda x: x[1], reverse=True)
    top_features[label] = [f[0] for f in sorted_features[:n]]

print(top_features)

In [ ]:
feature_names = ['credit amount', 'duration', 'age']
# Get the centroids
print(kmeans_final)

In [ ]:
weights = kmeans.cluster_centers_
print(weights)

# Create a bar chart for each cluster's feature weights
x_pos = np.arange(len(feature_names))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, feature_names)
    plt.ylabel('Weight')
    plt.title('Feature Weights for Cluster {}'.format(i+1))
    plt.show()

In [ ]:
male_count = []
female_count = []
n_clusters=3
for i in range(n_clusters):
  male_count.append(((clusters_config['Sex'] == 'male') & (clusters_config['cluster'] == i)).sum())
  female_count.append(((clusters_config['Sex'] == 'female') & (clusters_config['cluster'] == i)).sum())


weights = list(zip(male_count,female_count))
print(weights)
feature_names = ['male','female']
x_pos = np.arange(len(feature_names))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, feature_names)
    plt.ylabel('count')
    plt.title('Sex count for Cluster {}'.format(i+1))
    plt.show()


In [ ]:
weights = []
n_clusters=3
for i in range(n_clusters):
  weights.append((clusters_config['Age']).mean())



print(clusters_config)

"""
x_pos = np.arange(len(feature_names))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, feature_names)
    plt.ylabel('count')
    plt.title('Sex count for Cluster {}'.format(i+1))
    plt.show()
"""

In [ ]:


attributes_values = ['education','radio/TV','furniture/equipment','car']
list_of_attributes = []
#cria 
for i in range(len(attributes_values)):
  list_of_attributes.append([])


n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    list_of_attributes[j].append(((clusters_config['Purpose'] == attributes_values[j]) & (clusters_config['cluster'] == i)).sum())

print(list_of_attributes)

clusters = [[],[],[]]
n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    clusters[i].append(list_of_attributes[j][i])

print(clusters)
weights = clusters
x_pos = np.arange(len(attributes_values))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, attributes_values)
    plt.ylabel('count')
    plt.title('Purpose count for Cluster {}'.format(i+1))
    plt.show()



In [ ]:
free_count = []
own_count = []
n_clusters=3
for i in range(n_clusters):
  free_count.append(((clusters_config['Housing'] == 'free') & (clusters_config['cluster'] == i)).sum())
  own_count.append(((clusters_config['Housing'] == 'own') & (clusters_config['cluster'] == i)).sum())


weights = list(zip(free_count,own_count))
print(weights)
feature_names = ['free','own']
x_pos = np.arange(len(feature_names))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, feature_names)
    plt.ylabel('count')
    plt.title('Housing count for Cluster {}'.format(i+1))
    plt.show()


In [ ]:
attributes_values = ['little','moderate']
list_of_attributes = []
#cria 
for i in range(len(attributes_values)):
  list_of_attributes.append([])


n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    list_of_attributes[j].append(((clusters_config['Checking account'] == attributes_values[j]) & (clusters_config['cluster'] == i)).sum())

print(list_of_attributes)

clusters = [[],[],[]]
n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    clusters[i].append(list_of_attributes[j][i])

print(clusters)
weights = clusters
x_pos = np.arange(len(attributes_values))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, attributes_values)
    plt.ylabel('count')
    plt.title('Checking account for Cluster {}'.format(i+1))
    plt.show()


In [ ]:
attributes_values = ['little','moderate']
list_of_attributes = []
#cria 
for i in range(len(attributes_values)):
  list_of_attributes.append([])


n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    list_of_attributes[j].append(((clusters_config['Saving accounts'] == attributes_values[j]) & (clusters_config['cluster'] == i)).sum())

print(list_of_attributes)

clusters = [[],[],[]]
n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    clusters[i].append(list_of_attributes[j][i])

print(clusters)
weights = clusters
x_pos = np.arange(len(attributes_values))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, attributes_values)
    plt.ylabel('count')
    plt.title('Saving account for Cluster {}'.format(i+1))
    plt.show()


In [ ]:
attributes_values = [1,2,3]
list_of_attributes = []
#cria 
for i in range(len(attributes_values)):
  list_of_attributes.append([])


n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    list_of_attributes[j].append(((clusters_config['Job'] == attributes_values[j]) & (clusters_config['cluster'] == i)).sum())

print(list_of_attributes)

clusters = [[],[],[]]
n_clusters=3
for i in range(n_clusters):
  for j in range(len(attributes_values)):
    clusters[i].append(list_of_attributes[j][i])

print(clusters)
weights = clusters
x_pos = np.arange(len(attributes_values))
n_clusters=3
for i in range(n_clusters):
    plt.bar(x_pos, weights[i], align='center')
    plt.xticks(x_pos, attributes_values)
    plt.ylabel('count')
    plt.title('Jobs for Cluster {}'.format(i+1))
    plt.show()

In [ ]:
mean_age = df.groupby('cluster')['Age'].mean()
print(mean_age)
mean_age = df.groupby('cluster')['Credit amount'].mean()
print(mean_age)
mean_age = df.groupby('cluster')['Duration'].mean()
print(mean_age)
